___
#### 0401

## Overview
- groupby로 그룹을 생성
- 그룹 : 범주형(categorical) 데이터를 이용 ; 연속형 변수 : 양자화해서 사용
- agg/ transform / apply 메소드로 그룹 연산
- 연속형 변수를 양자화
- pd.pivot_table 함수 : pivot + groupby
- pd.crosstab 
- 실습 : 주가 데이터, 대통령 후보 기부 데이터

In [3]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4)

In [ ]:
path = 'C:/Users/tmznq/workspace/ml_scratch/seoul_coding_academy/pydata-book-1st-edition/ch08/tips.csv'

tips = pd.read_csv(path)

### groupby, transform

In [28]:
# DataFrame 연습
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [30]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')
k1_means

,mean_data1,mean_data2
key1,,
a,0.746672,0.910916
b,-0.537585,0.525384


In [31]:
people = DataFrame(np.random.randn(5, 5),
                   columns=['a', 'b', 'c', 'd', 'e'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.ix[2:3, ['b', 'c']] = np.nan # Add a few NA values
people

C:\Users\tmznq\Anaconda3\envs\ml_scratch\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


,a,b,c,d,e
Joe,1.007189,-1.296221,0.274992,0.228913,1.352917
Steve,0.886429,-2.001637,-0.371843,1.669025,-0.438570
Wes,-0.539741,NaN,NaN,-1.021228,-0.577087
Jim,0.124121,0.302614,0.523772,0.000940,1.343810
Travis,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


In [34]:
keys = ['one', 'two', 'one', 'two', 'one']
people.groupby(keys).mean()

,a,b,c,d,e
one,-0.082032,-1.063687,-1.047620,-0.884358,-0.028309
two,0.505275,-0.849512,0.075965,0.834983,0.452620


- 각 그룹에서 평균값을 빼고 싶으면?

In [35]:
def demean(arr):
    return arr - arr.mean()

# 평균을 0으로 돌려놓고 분산을 만들어주기 위해

In [37]:
demeaned = people.groupby(keys).transform(demean)
demeaned

,a,b,c,d,e
Joe,1.089221,-0.232534,1.322612,1.113271,1.381226
Steve,0.381154,-1.152125,-0.447807,0.834043,-0.891190
Wes,-0.457709,NaN,NaN,-0.136869,-0.548778
Jim,-0.381154,1.152125,0.447807,-0.834043,0.891190
Travis,-0.631512,0.232534,-1.322612,-0.976402,-0.832448


- demeaned 객체 그룹의 평균은 0이 되어야 한다.

In [38]:
demeaned.groupby(keys).mean()

,a,b,c,d,e
one,0.000000e+00,-1.110223e-16,0.0,7.401487e-17,0.0
two,-2.775558e-17,0.000000e+00,0.0,0.000000e+00,0.0


### apply 함수 적용
- 행을 선택하기위해 사용할 때. 여기서 aggregate나 transform을 사용하기 제한된다.
- 변환을 적용할 데이터는 1개만 가능?

In [39]:
def top(df, n=5, column = ''):
    return df.sort_values(by=column)[-n:]

- groupby 객체에 describe 함수를 호출하면 내부적으로 다음과 같은 단계를 거친다  
f = lambda x: x.describe();grouped.apply(f)

In [40]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [41]:
result = tips.groupby('smoker')['tip_pct'].agg(['count', 'mean', 'std', 'min', 'max'])
result

,count,mean,std,min,max
smoker,,,,,
No,151,0.159328,0.039910,0.056797,0.291990
Yes,93,0.163196,0.085119,0.035638,0.710345


- 내부적으로 count,mean 등의 함수가 agg 함수에 전달되었음
- agg에 전달한 여러 함수들은 내포된 열 색인을 생성함.
- 따라서 열 이름이 생성된 것으로 이해하자.

### 양자화
- cut(등구간), qcut(등순위)
- 등구간을 지정하지 않으려면 edge point를 주면 됨.

In [ ]:
frame = DataFrame({'data1': np.random.randn(1000),
                   'data2': np.random.randn(1000)})
factor = pd.cut(frame.data1, 4)
factor[:10]

In [ ]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(factor)
grouped.apply(get_stats).unstack()

#ADAPT the output is not sorted in the book while this is the case now (swap first two lines)

In [ ]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)

grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

### Filling missing values wiht group-specific values(결측치 관리)
- fillna 함수로 group 안의 결측치 대체

In [4]:
s = Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    0.478943
2         NaN
3   -0.555730
4         NaN
5    1.393406
dtype: float64

In [5]:
s.fillna(s.mean())

0    0.438873
1    0.478943
2    0.438873
3   -0.555730
4    0.438873
5    1.393406
dtype: float64

In [ ]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']

In [11]:
group_key = ['East'] * 4 + ['West'] * 4
data = Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          3.248944
New York     -1.021228
Vermont            NaN
Florida       0.124121
Oregon        0.302614
Nevada             NaN
California    0.000940
Idaho              NaN
dtype: float64

In [8]:
data.groupby(group_key).mean()

East    0.622501
West    0.564642
dtype: float64

In [9]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          1.352917
New York      0.886429
Vermont       0.622501
Florida      -0.371843
Oregon        1.669025
Nevada        0.564642
California   -0.539741
Idaho         0.564642
dtype: float64

- apply 함수가 아닌 transform 함수를 적용해도 동일하다.
- 위의 경우, fillna 함수 자체가 filtering 기능을 가지므로 transform이 더 적절하다. why?

In [10]:
fill_values = {'East': 0.5, 'West': -1} # 그룹에 따라 채워넣을 값을 미리 정함.
fill_func = lambda g: g.fillna(fill_values[g.name])

data.groupby(group_key).apply(fill_func)

Ohio          1.352917
New York      0.886429
Vermont       0.500000
Florida      -0.371843
Oregon        1.669025
Nevada       -1.000000
California   -0.539741
Idaho        -1.000000
dtype: float64

### Example: Random sampling and permutation (랜덤표본과 순열)
- 머신러닝 모델을 돌리기 위한 data의 train과 test를 나누기 위해 사용함.
- permutation : 순열

In [21]:
# 트럼프 카드 만들기

# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']

# values
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']

# index
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = Series(card_val, index=cards)

In [22]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

- random sampling

In [23]:
def draw(deck, n=5):
    return deck.take(np.random.permutation(len(deck))[:n])
draw(deck)

KS     10
10D    10
5H      5
KC     10
9C      9
dtype: int64

In [ ]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

In [ ]:
# alternatively
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

___
## 실습
- tips 데이터에서 grouped 생성
- 연속형 변수들(bill, tip 등)은 mean, 범주형 변수는 count를 적용  
___
1. agg
2. apply
    - 필요 시 concat를 사용
3. pivot_table
    - 필요 시 concat 사용
    - pivot_table 사용 시, 중간합계를 계산한다.

In [27]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
